# Failure Case Analysis
Visualise the misclasified samples.

## Load libraries and files

In [23]:
import os

import numpy as np
import pandas as pd
import seaborn


from pathlib import Path

from matplotlib import pyplot as plt

# Print information about the current working directory
print(os.getcwd())

/home/kevin/DPhil/Projects/EHR-Indication-Processing/03_Evaluation


In [24]:
data_path = Path("../00_Data/export")

model_name = "Bio_ClinicalBERT"

bert_predictions_path = data_path/f"{model_name}_predictions.csv"
bert_true_path = data_path/f"{model_name}_true_labels.csv"

bert_true = pd.read_csv(bert_true_path)
bert_predictions = pd.read_csv(bert_predictions_path)

# Set indicies
bert_true = bert_true.set_index('IndicationRaw')
bert_predictions = bert_predictions.set_index(bert_true.index)

# Convert to booleans
bert_true = bert_true.astype(int)
bert_predictions = bert_predictions.astype(int)

## Extract the failure cases

In [25]:
reference_column = "Reference"
predicted_column = "Predicted"
compared = bert_true.compare(bert_predictions, result_names=(reference_column, predicted_column), keep_equal=True).dropna(how='all')

In [26]:
def highlight_differences(s):
    row_style = []
    highlight_format_true = "color:green; font-weight:bold"
    highlight_format_false = "color:red; font-weight:bold"

    # Iterate over the first index (true labels) and set colour if not identical
    for level_value in s.index.get_level_values(0).unique():
        level_slice = s.loc[level_value, :]
        if level_slice[reference_column] == level_slice[predicted_column]:
            row_style += [None, None]
        else:
            row_style += [highlight_format_true, highlight_format_false]
    
    return row_style

compared_highlighted = compared.style.apply(highlight_differences, axis=1)

## Export the analysis

In [27]:
compared_highlighted_path = data_path/f"{model_name}_compared.xlsx"
compared_highlighted.to_excel(compared_highlighted_path)